# Elixir LiveBook on Google Colab w/ BumbleeBee + GPU support
### For support, contact [Luke Galea](https://github.com/lukegalea)

Runs Livebook w/ Bumblebee pre-installed with a local tunnel so you can use the web UI from your local browser.

<hr>

### Step 1: Choose your runtime type

Decide if you want to run ML on CPU, TPU or GPU. Choose Normal or High RAM and Normal or Premium GPU.

Go to Colab Menu -> Runtime -> Choose runtime type and configure accordingly.

**

### Step 2: Choose your XLA backend
Choose XLA backend to match below under "Configuration". If using GPU, choose "cuda".

**

### Step 3: Launch Livebook
Go to Colab Menu -> Runtime -> Run All

It will take 5-10 mins to launch. When ready, note the password and tunnel url. Open the tunnel url in a new browser window, enter password and you are good to go!

# Configuration

In [ ]:
XLA_Backend = "cuda" #@param ["cpu", "tpu", "cuda"]

# Setup

In [ ]:
#@markdown #Update System
!sudo apt update
!sudo apt upgrade
!sudo apt-get install xkcdpass libsctp1 build-essential git wget libssl-dev libreadline-dev libncurses5-dev zlib1g-dev m4 curl wx-common autoconf libxml2-utils xsltproc fop unixodbc unixodbc-dev libncurses5 libwxgtk3.0-gtk3-0v5

In [ ]:
!sudo apt --fix-broken install

In [ ]:
#@markdown #Install Erlang
!wget https://packages.erlang-solutions.com/erlang/debian/pool/esl-erlang_25.0.4-1~ubuntu~bionic_amd64.deb
!sudo dpkg -i esl-erlang_25.0.4-1~ubuntu~bionic_amd64.deb


In [ ]:
#@markdown #Install Elixir
!wget https://github.com/elixir-lang/elixir/releases/download/v1.14.2/elixir-otp-25.zip
!mkdir elixir && mv elixir-otp-25.zip elixir/ && cd elixir && unzip elixir-otp-25.zip

In [ ]:
#@markdown #Add Elixir to Path
import os
os.environ['PATH'] += ':/content/elixir/bin/'

In [ ]:
#@markdown #Update Mix
!mix local.hex --force
!mix do local.rebar --force

In [ ]:
#@markdown #Install Local Tunnel
!npm install -g localtunnel

In [ ]:
#@markdown #Setup Rustup
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- --default-toolchain none -y

In [ ]:
#@markdown #Add Rustup to Path
import os
os.environ['PATH'] += ':/root/.cargo/bin/'

In [ ]:
#@markdown #Setup Rust
!rustup toolchain install nightly --allow-downgrade --profile minimal --component clippy

In [ ]:
#@markdown #Install Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/download/v1.15.0/bazelisk-linux-amd64
!mv bazelisk-linux-amd64 bazel
!chmod u+x bazel
!mkdir /root/bin/
!mv bazel /root/bin/

In [ ]:
#@markdown #Add Bazel to Path
import os
os.environ['PATH'] += ':/root/bin/'

In [ ]:
#@markdown #Install Bazel
!bazel

In [ ]:
#@markdown #Setup XLA
if XLA_Backend == 'cpu':
  %set_env XLA_BUILD=false
  %set_env XLA_TARGET=cpu
  %set_env XLA_ARCHIVE_URL=http://ideaforge.org/xla_extension-x86_64-linux-gnu-cpu.tar.gz
  %set_env TOKENIZERS_BUILD=true

if XLA_Backend == 'cuda':
  %set_env XLA_BUILD=false
  %set_env XLA_TARGET=cuda  
  %set_env XLA_ARCHIVE_URL=http://ideaforge.org/xla_extension-x86_64-linux-gnu-cuda.tar.gz
  %set_env TOKENIZERS_BUILD=true

# I haven't gotten around to building this yet so no precompiled archive
if XLA_Backend == 'tpu':
  %set_env XLA_BUILD=true
  %set_env XLA_TARGET=tpu    
  %set_env TOKENIZERS_BUILD=true

In [ ]:
#@markdown #Setup Livebook
!git clone https://github.com/livebook-dev/livebook.git

## Edit config.exs and mix.exs for bumblebee
# To mix.exs
# {:bumblebee, "~> 0.1.2"},
# {:exla, ">= 0.0.0"},
# {:rustler, ">= 0.0.0"}
# {:rustler_precompiled, "~> 0.5"}
# {:kino_bumblebee, "~> 0.1.4"}

# To config.ex
# config :nx, default_backend: EXLA.Backend
# config :rustler_precompiled, :force_build, libex_tokenizers: true

!sed -i -e 's/{:bypass, "~> 2.1", only: :test},/{:bypass, "~> 2.1", only: :test},/g' -e '/{:bypass, "~> 2.1", only: :test},/a {:bumblebee, "~> 0.1.2"}, {:exla, ">= 0.0.0"}, {:rustler, ">= 0.0.0"}, {:rustler_precompiled, "~> 0.5"}, {:kino_bumblebee, "~> 0.1.4"}' livebook/mix.exs
!sed -i -e '/import Config/a config :nx, default_backend: EXLA.Backend' livebook/config/config.exs 
!sed -i -e 's/port: 8080/port: 9090/g' livebook/config/prod.exs

In [ ]:
# patch livebook/mix.exe to install dependencies for nx, bumblebee, etc.

import urllib.request 
urllib.request.urlretrieve("https://raw.githubusercontent.com/yusefmosiah/LiveBook_GoogleColab/main/mix.exs", "/content/livebook/mix.exs")
urllib.request.urlretrieve("https://raw.githubusercontent.com/yusefmosiah/LiveBook_GoogleColab/main/config.exs", "/content/livebook/config/config.exs")

In [ ]:
# first edit livebook mix.exs lines 106-111 for syntax (missing comma)

!cd livebook && mix deps.get --only prod
!cd livebook && MIX_ENV=prod mix deps.compile --only prod

# Run


In [ ]:
# Generate password
password = !xkcdpass -n 2 -d-
password = password[0]
print(f"Password is {password}")

# Launch Localtunnel
import os
import time
import sys
from subprocess import getoutput

!nohup lt --port 9090 > srv.txt 2>&1 &
time.sleep(3)
srv= getoutput('cat /content/srv.txt')
print(srv)

In [ ]:
!cd livebook && LIVEBOOK_DEFAULT_RUNTIME=embedded LIVEBOOK_PASSWORD=$password MIX_ENV=prod mix phx.server